In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import time

from lightglue import LightGlue, SuperPoint, viz2d, match_pair
from lightglue.utils import rbd

from lac.slam.feature_tracker import FeatureTracker
from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data, grayscale_to_3ch_tensor
from lac.params import LAC_BASE_PATH, DT

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path("/home/shared/data_raw/LAC/segmentation/slam_map1_preset1_teleop")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

In [ ]:
left_imgs = {}
right_imgs = {}
side_left_imgs = {}
side_right_imgs = {}

for img_name in tqdm(os.listdir(data_path / "FrontLeft")):
    left_imgs[int(img_name.split(".")[0])] = cv2.imread(str(data_path / "FrontLeft" / img_name), cv2.IMREAD_GRAYSCALE)

for img_name in tqdm(os.listdir(data_path / "FrontRight")):
    right_imgs[int(img_name.split(".")[0])] = cv2.imread(str(data_path / "FrontRight" / img_name), cv2.IMREAD_GRAYSCALE)

for img_name in tqdm(os.listdir(data_path / "Left")):
    side_left_imgs[int(img_name.split(".")[0])] = cv2.imread(str(data_path / "Left" / img_name), cv2.IMREAD_GRAYSCALE)
for img_name in tqdm(os.listdir(data_path / "Right")):
    side_right_imgs[int(img_name.split(".")[0])] = cv2.imread(str(data_path / "Right" / img_name), cv2.IMREAD_GRAYSCALE)

assert len(left_imgs.keys()) == len(right_imgs.keys())
img_idxs = sorted(left_imgs.keys())

In [ ]:
tracker = FeatureTracker(cam_config)

image = side_left_imgs[1500]

feats = tracker.extract_feats(image)
feats = rbd(feats)

In [ ]:
kps = feats["keypoints"]
good_kps = kps[feats["keypoint_scores"] > 0.05]
print(f"Num keypoints: {len(kps)}, {len(good_kps)}")

In [ ]:
viz2d.plot_images([image])
viz2d.plot_keypoints([kps], ps=10)
viz2d.plot_keypoints([good_kps], colors=["red"], ps=10)

# LightGlue Tracking


In [ ]:
# Frame indices
idx0 = 1500
idx1 = idx0 + 6

prev_img = side_right_imgs[idx0]
next_img = side_right_imgs[idx1]

prev_feats = tracker.extract_feats(prev_img)
next_feats = tracker.extract_feats(next_img)

matches = tracker.match_feats(prev_feats, next_feats)
points_prev = prev_feats["keypoints"][0][matches[:, 0]]
points_next = next_feats["keypoints"][0][matches[:, 1]]

print(len(matches))

matches = tracker.match_feats(prev_feats, next_feats, min_score=0.9)
print(len(matches))

In [ ]:
points0 = points_prev.cpu().numpy()
points1 = points_next.cpu().numpy()

plt.figure(figsize=(10, 6))
plt.imshow(next_img, cmap="gray")
for i in range(len(matches)):
    plt.plot([points0[i, 0], points1[i, 0]], [points0[i, 1], points1[i, 1]], color="lime")
    plt.scatter(points1[i, 0], points1[i, 1], color="lime", s=5)
plt.axis("off")
plt.show()

plt.figure(figsize=(10, 6))
plt.imshow(prev_img, cmap="gray")
for i in range(len(matches)):
    plt.plot([points0[i, 0], points1[i, 0]], [points0[i, 1], points1[i, 1]], color="lime")
    plt.scatter(points0[i, 0], points0[i, 1], color="lime", s=5)
plt.axis("off")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv

from lac.utils.frames import get_cam_pose_rover, CAMERA_TO_OPENCV_PASSIVE
from lac.perception.vision import get_camera_intrinsics

# Camera intrinsics and extrinsics
K = get_camera_intrinsics("Right", cam_config)
rover_T_cam = get_cam_pose_rover("Right")
rover_T_cam_ocv = rover_T_cam.copy()
rover_T_cam_ocv[:3, :3] = rover_T_cam_ocv[:3, :3] @ CAMERA_TO_OPENCV_PASSIVE

# Projection matrices
cam_T_world_0 = inv(poses[idx0] @ rover_T_cam_ocv)
cam_T_world_1 = inv(poses[idx1] @ rover_T_cam_ocv)

P0 = K @ cam_T_world_0[:3]
P1 = K @ cam_T_world_1[:3]

# Triangulate
points_4d_h = cv2.triangulatePoints(P0, P1, points0.T, points1.T)
points_3d_est = (points_4d_h[:3] / points_4d_h[3]).T

# Estimated depths
depths_est = (cam_T_world_0[:3, :3] @ points_3d_est.T + cam_T_world_0[:3, 3:4]).T[:, 2]


plt.figure(figsize=(10, 6))
plt.imshow(next_img, cmap="gray")
for i in range(len(points1)):
    plt.plot([points0[i, 0], points1[i, 0]], [points0[i, 1], points1[i, 1]], color="lime")
    x, y = points1[i]
    plt.scatter(x, y, color="lime", s=5)
    plt.text(x + 2, y, f"{depths_est[i]:.2f}", color="red", fontsize=8)
plt.axis("off")
plt.show()

In [ ]:
points_2d_0, points_2d_1

In [ ]:
from plotly import graph_objects as go
from lac.utils.plotting import plot_poses

idx0 = 1500
idx1 = idx0 + 10

# Synthetic 3D points in camera frame 0
points_cam = np.array(
    [
        [0.0, 0.0, 2.0],
        [-1.0, 0.0, 3.0],
        [0.0, -1.5, 4.0],
    ]
)
points_cam_h = np.hstack((points_cam, np.ones((points_cam.shape[0], 1))))
points_world = (poses[idx0] @ rover_T_cam_ocv @ points_cam_h.T).T

# World to camera transforms
cam_T_world_0 = np.linalg.inv(poses[idx0] @ rover_T_cam_ocv)
cam_T_world_1 = np.linalg.inv(poses[idx1] @ rover_T_cam_ocv)

points_world_h = np.hstack((points_world[:, :3], np.ones((points_world.shape[0], 1))))
points_cam_0 = (cam_T_world_0 @ points_world_h.T).T[:, :3]
points_cam_1 = (cam_T_world_1 @ points_world_h.T).T[:, :3]

# Project to 2D image points
points_2d_0 = (K @ points_cam_0.T).T
points_2d_0 = (points_2d_0[:, :2].T / points_2d_0[:, 2]).T

points_2d_1 = (K @ points_cam_1.T).T
points_2d_1 = (points_2d_1[:, :2].T / points_2d_1[:, 2]).T

# Projection matrices
P0 = K @ cam_T_world_0[:3]
P1 = K @ cam_T_world_1[:3]

# Triangulate
points_4d_h = cv2.triangulatePoints(P0, P1, points_2d_0.T, points_2d_1.T)
points_3d_est = (points_4d_h[:3] / points_4d_h[3]).T

# Estimated depths
depths_est = (cam_T_world_0[:3, :3] @ points_3d_est.T + cam_T_world_0[:3, 3:4]).T[:, 2]

# Ground truth depths
gt_depths = points_cam_0[:, 2]

print(f"GT depths: {gt_depths}")
print(f"Estimated depths: {depths_est}")

# Optional: visualize
fig = go.Figure()
plot_poses([poses[idx0], poses[idx1]], fig)
plot_poses([poses[idx0] @ rover_T_cam_ocv], fig)
plot_poses([poses[idx1] @ rover_T_cam_ocv], fig)
fig.add_scatter3d(
    x=points_world[:, 0],
    y=points_world[:, 1],
    z=points_world[:, 2],
    mode="markers",
    marker=dict(size=5, color="red"),
)

In [ ]:
depths_est

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(next_img, cmap="gray")
for i in range(len(points_2d_1)):
    plt.plot([points_2d_0[i, 0], points_2d_1[i, 0]], [points_2d_0[i, 1], points_2d_1[i, 1]])
    plt.scatter(points_2d_0[i, 0], points_2d_0[i, 1], s=5, color=f"C{i}")
    plt.scatter(points_2d_1[i, 0], points_2d_1[i, 1], s=20, color=f"C{i}")
    plt.text(points_2d_1[i, 0] + 2, points_2d_1[i, 1], f"{depths_est[i]:.2f}", color="red", fontsize=8)